In [ ]:
import requests,json,config,pickle

coordinate=44.799432, 20.447455
x=44.01115533984336
y=20.910337884603837
radius=300
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={x}%2C{y}&radius={radius}&type=restaurant&key={config.api_key}"

payload={}
headers ={}


l=[]

In [107]:
class VCrequestClass:
    api_key=""
    payload={}
    headers ={}
    def __init__(self):
        self.api_key=config.api_key
    def _glupi_hash(self,x,y,radius,type):
        return str(x)+'|'+str(y)+str(radius)+'cafe'
    def request(self,x,y,radius,type):
        time.sleep(2)
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={x}%2C{y}&radius={radius}&type={type}&key={self.api_key}"
        with open('cache.pkl', 'rb') as f:
            cache = pickle.load(f)
        if(self._glupi_hash(x,y,radius,type) in cache):
            print('uzeto iz cache')
            return cache[self._glupi_hash(x,y,radius,type)]
        else:
            page = requests.request("GET", url, headers=self.headers, data=self.payload)
            page_dict=json.loads(page1.text)
            cache[self._glupi_hash(x,y,radius,type)]=page_dict
            print(cache.keys())
            with open('cache.pkl', 'wb') as f:
                pickle.dump(cache, f)
            return page_dict
       


In [ ]:
import time
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={x}%2C{y}&radius={radius}&type=cafe&key={config.api_key}"

page1 = requests.request("GET", url, headers=headers, data=payload)
page1_dict=json.loads(page1.text)
l+=page1_dict['results']
broj=1
if('next_page_token' in page1_dict):
    broj=2
    next_page=page1_dict['next_page_token']
    time.sleep(2)
    url2 = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page}&key={config.api_key}"
    page2 = requests.request("GET", url2, headers=headers, data=payload)
    page2_dict=json.loads(page2.text)
    l+=page2_dict['results']
    if('next_page_token' in page2_dict):
        broj=3
        time.sleep(2)
        next_page=page2_dict['next_page_token']
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page}&key={config.api_key}"
        page3 = requests.request("GET", url, headers=headers, data=payload)
        page3_dict=json.loads(page3.text)
        l+=page3_dict['results']


In [ ]:
for i in l: 
    # print(f"{i['name']}")
    # print(f"{i}")
    print(i['geometry']['location']['lat'],i['geometry']['location']['lng'])

In [ ]:
len(l)

print("Cao cofi")


In [ ]:
from ipyleaflet import *
from ipywidgets import HTML

m=Map(center=(x,y),zoom=12,scroll_wheel_zoom=True)

for i in l:
    marker = Marker(location=(i['geometry']['location']['lat'],i['geometry']['location']['lng']),draggable=False)
    m.add_layer(marker)
   
    message = HTML()
    message.value = i['name']
    
    popup = Popup(
        location=(x,y),
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )

    marker.popup = message

circle3 = Circle()
circle3.location = (44.01115533984336, 20.910337884603837)
circle3.radius = 300
circle3.color = "red"
circle3.fill_color = "red"
circle2 = Circle()
circle2.location = (44.11115533984336, 20.910337884603837)
circle2.radius = radius
circle2.color = "red"
circle2.fill_color = "red"
print(circle2)
print(circle3)
m.add_layer(circle2)
m.add_layer(circle3)

m